## Python imports

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from jinja2 import Template
import os
import sys

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

### _Obtain App and DB models_

In [ ]:
from django.apps import apps
app_models = [v for k,v in apps.all_models['bookstore_api'].items()]

dbdesign = [{'model': model._meta.object_name, \
             'fields': [field.name for field in \
                        model._meta.get_fields()]} \
            for model in app_models]

## Adding Views

In [ ]:
from django.apps import apps
file = '/bookstore/bookstore_api/views.py'  
    
with open(file, 'w') as file:
    template = Template(open('/templates/views.j2', 'r').read())
    print(template.render(dbdesign=[app['model'] for app in dbdesign]))
    file.write(template.render(dbdesign=[app['model'] for app in dbdesign]))

## Adding URLs

In [ ]:
from django.apps import apps
file = '/bookstore/bookstore_api/urls.py' 
    
with open(file, 'w') as file:
    template = Template(open('/templates/urls_app.j2', 'r').read())
    print(template.render(dbdesign=[app['model'] for app in dbdesign]))
    file.write(template.render(dbdesign=[app['model'] for app in dbdesign]))

In [ ]:
from django.apps import apps
file = '/bookstore/project/urls.py'   
appname = [app for app in apps.app_configs][-1]

with open(file, 'w') as file:
    template = Template(open('/templates/urls.j2', 'r').read())
    print(template.render(app_name=appname))
    file.write(template.render(app_name=appname))

### _let's check . . ._

In [ ]:
url = 'http://bookstore:8000'

def page_title(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

page_title(url)

![title](images/drf.png)

## JSON browsing

In [ ]:
url = 'http://bookstore:8000'
headers = {'Content-type': 'application/json'}
requests.get(url, headers=headers).json()

In [ ]:
for model in [app['model'] for app in dbdesign]:
    response = requests.get(url + "/%s" % model.lower(), headers=headers).json()
    print("A total of %s %s API Objects\n" % (len(response),model))
    print(json.dumps(response[-2:], indent=4, sort_keys=True))